#### Calculating the percent share of agriculture in total employment per municipality or province for the year 2022 (PSA Labor Force Survey)

Step 1: Import pandas

In [ ]:
import pandas as pd

Step 2: Combine all csv files in folder into one DF

In [ ]:
# Import glob module to find files in directory
import glob

In [ ]:
# Store all csv filenames in list
file_list = glob.glob(r"L*.csv")
file_list

In [ ]:
# Create empty dataframe to store all combined DFs
lfs_2022_df = pd.DataFrame()

# Define function to check if column name...
# ...contains 'REG', 'WORK', 'PROVMUN', or 'PKB'
def check_str(col_name):
     substrs = ['REG','_WORK', '_PROVMUN', '_PKB']
     return any(x in col_name for x in substrs)

# For each csv file in folder...
for fp in file_list:
    # ...read into a DF, w/ specified cols & NaN values...
    df = pd.read_csv(fp, usecols=check_str,
                    na_values=[' ', '  ', '   ', '    ', '     ', '      '])
    
    # ...then append to empty DF for whole year
    lfs_2022_df = pd.concat([lfs_2022_df, df], ignore_index=True)
    
# Display head of merged DF
lfs_2022_df.head(20)

Step 3: Merge all alike columns and drop extra columns

In [ ]:
# Examine first 20 rows where PUFC11_WORK is not null
lfs_2022_df.loc[~lfs_2022_df['PUFC11_WORK'].isnull()].head(20)

In [ ]:
# For all rows where PUFC11_WORK is null, copy values from PUFC09_WORK and PUFC09A_WORK
lfs_2022_df.loc[lfs_2022_df['PUFC11_WORK'].isnull(), 'PUFC11_WORK'] =\
    lfs_2022_df['PUFC09_WORK']

lfs_2022_df.loc[lfs_2022_df['PUFC11_WORK'].isnull(), 'PUFC11_WORK'] =\
    lfs_2022_df['PUFC09A_WORK']

lfs_2022_df.tail(10)

In [ ]:
# Drop extra WORK columns
lfs_2022_df = lfs_2022_df.drop(['PUFC09_WORK', 'PUFC09A_WORK'], axis=1)
lfs_2022_df.tail(10)

Repeat Step 3 for PROVMUN and PKB columns

In [ ]:
# For all rows where PUFC12A_PROVMUN is null, copy values from PUFC11A_PROVMUN
lfs_2022_df.loc[lfs_2022_df['PUFC12A_PROVMUN'].isnull(), 'PUFC12A_PROVMUN'] =\
    lfs_2022_df['PUFC11A_PROVMUN']

lfs_2022_df.tail(10)

In [ ]:
# For all rows where PUFC6_PKB is null, copy values from PUFC15_PKB
lfs_2022_df.loc[lfs_2022_df['PUFC16_PKB'].isnull(), 'PUFC16_PKB'] =\
    lfs_2022_df['PUFC15_PKB']

lfs_2022_df.tail(10)

In [ ]:
# Drop extra PROVMUN and PKB columns
lfs_2022_df = lfs_2022_df.drop(['PUFC11A_PROVMUN', 'PUFC15_PKB'], axis=1)
lfs_2022_df.tail(10)

Step 4: Save to new DF only the rows where WORK = 1 (employed)

In [ ]:
work_df = lfs_2022_df.loc[lfs_2022_df['PUFC11_WORK'] == 1]
work_df.sample(20)

Step 5: Rename columns and drop work indicator column

In [ ]:
col_names = {
    'PUFREG': 'REGION',
    'PUFC12A_PROVMUN': 'PROV_MUN',
    'PUFC16_PKB': 'INDUSTRY'
}

work_df = work_df.rename(columns=col_names)[['REGION', 'PROV_MUN', 'INDUSTRY']]
work_df.sample(20)

Step 6: Parse province out of PROV_MUN column

In [ ]:
# Remove decimal place from prov-muni by converting to int
work_df['PROV_MUN'] = work_df['PROV_MUN'].astype(int)

# Convert to string, pad w/ leading zeroes up to 4 chars,
# then slice out first 2 chars as province code
work_df['PROVINCE'] = work_df['PROV_MUN'].astype(str)\
                      .str.zfill(4).str.slice(0,2)

work_df.sample(20)

In [ ]:
# Count number of unique municipality codes
len(work_df['PROV_MUN'].unique())

In [ ]:
# Count number of unique province codes
len(work_df['PROVINCE'].unique())

Step 7: Group DF by municipality

In [ ]:
by_muni = work_df.groupby('PROV_MUN')
len(by_muni)

Step 8: Calculate percent share of agriculture in total employment per municipality

In [ ]:
# Create empty dataframe for aggregated values
agshare_by_muni = pd.DataFrame()

# Iterate over ach municipality
for key, group in by_muni:

    # Create empty series to store each calculation
    c = pd.Series()

    # Add region & muni code to series of values
    c['REGION'] = group['REGION'].median().astype(int)
    c['PROVINCE'] = group['PROVINCE'].astype(int).median()
    c['PROV_MUN'] = key

    # Count total number of employed (i.e. length per group)
    c['TOTAL_EMPLOYED'] = group['INDUSTRY'].count()

    # Count number of people employed in agriculture
    c['AGRI_EMPLOYED'] = group.loc[group['INDUSTRY'] < 4, 'INDUSTRY'].count()

    # Calculate ratio between agri & total employment
    # and round off values to 2 decimal places
    c['PERCENT_AGRI'] = c['AGRI_EMPLOYED'] / c['TOTAL_EMPLOYED'] * 100
    c['PERCENT_AGRI'] = round(c['PERCENT_AGRI'], 2)

    # Convert series into dataframe and transpose into a row
    row = c.to_frame().transpose()

    # Append new row into agshare_by_muni dataframe
    agshare_by_muni = pd.concat([agshare_by_muni, row], ignore_index=True)

agshare_by_muni

Step 9: Read metadata file into DF of province & municipality names

In [ ]:
# Read metadata xlsx file as DF

names_fp = r"lfs_november_2022_metadata(dictionary).xlsx"

geo_names = pd.read_excel(names_fp, sheet_name=r"lfs_november_2022_valueset",
                          skiprows=168, skipfooter=284)

# Define dict to rename columns
new_cols = {
    'Unnamed: 2': 'LOCATION',
    'Unnamed: 3': 'LOC_CODE'
}

geo_names = geo_names.rename(columns=new_cols)[['LOCATION', 'LOC_CODE']]

geo_names.sample(10)

In [ ]:
# Split LOCATION column into province and municipality
names = geo_names['LOCATION'].str.split(' - ', expand=True)

# Put first item of split list into PROV_NAME column
# But only get all chars after 4-digit code and space
geo_names['PROV_NAME'] = names[0].str.slice(start=6)

# Put second item of split list into MUN_NAME column
geo_names['MUN_NAME'] = names[1]

geo_names.sample(10)


In [ ]:
# Drop unneeded LOCATION column
geo_names.drop(columns=['LOCATION'], inplace=True)

geo_names.sample(10)

Step 10: Join names DF with employment DF (on muni code)

In [ ]:
agshare_by_muni = agshare_by_muni.merge(right=geo_names, left_on='PROV_MUN', right_on='LOC_CODE')

agshare_by_muni.sample(10)

Step 11: Clean up province and municipality names

In [ ]:
# Remove parentheticals from province & municipality names
# agshare_by_muni['PROV_NAME'] = agshare_by_muni['PROV_NAME'].str.replace(r"\(.+\)", "")

aliases = agshare_by_muni['PROV_NAME'].str.split(' \(', expand=True)
agshare_by_muni['PROV_NAME'] = aliases[0].str.strip()

list(agshare_by_muni['PROV_NAME'].unique())

In [ ]:
# Do the same for municipality names
aliases = agshare_by_muni['MUN_NAME'].str.split(' \(', expand=True)
agshare_by_muni['MUN_NAME'] = aliases[0].str.strip()

agshare_by_muni['MUN_NAME'].head(20)

Step 9B: Clean up some rows

In [ ]:
# Define dict w/ keys = old names and values = fixed names
city_names = {
    'Calaca': 'City of Calaca',
    'Pres. Carlos P. Garcia': 'President Carlos P. Garcia',
    'Baliuag': 'City of Baliwag',
    'Carmona': 'City of Carmona',
    'Datu Saudi-Ampatuan': 'Datu Saudi Ampatuan',
    'Pio V. Corpuz': 'Pio V. Corpus',
    'Bacungan': 'Leon T. Postigo',
    'Hinoba-an': 'Hinoba-An'
}

agshare_by_muni.replace({'MUN_NAME': city_names}, inplace=True)

new_names = list(city_names.values())

agshare_by_muni.loc[agshare_by_muni['MUN_NAME'].isin(new_names)]

In [ ]:
# Assign municipality names for City of Isabela and City of Cotabato
agshare_by_muni.loc[agshare_by_muni['PROV_NAME'] == 'CITY OF ISABELA', 'MUN_NAME'] \
    = 'City of Isabela'

agshare_by_muni.loc[agshare_by_muni['PROV_NAME'] == 'COTABATO CITY', 'MUN_NAME'] \
    = 'City of Cotabato'

agshare_by_muni.loc[agshare_by_muni['MUN_NAME'].isin(['City of Isabela', 'City of Cotabato'])]

In [ ]:
# Merge all Manila sub-municipalities into one municipality (City of Manila)
manila = agshare_by_muni.loc[agshare_by_muni['PROVINCE'] == 39]\
        .groupby('PROVINCE', as_index=False)\
        .agg({
            'REGION': 'min',
            'PROVINCE': 'min',
            'PROV_MUN': 'min',
            'TOTAL_EMPLOYED': 'sum',
            'AGRI_EMPLOYED': 'sum',
            'PERCENT_AGRI': 'min',
            'LOC_CODE': 'min',
            'PROV_NAME': 'first',
            'MUN_NAME': 'first'
        })

# Re-calculate PERCENT_AGRI and assign municipality name
manila['PERCENT_AGRI'] = round(manila['AGRI_EMPLOYED'] / manila['TOTAL_EMPLOYED'] * 100, 2)
manila['MUN_NAME'].replace({'Tondo I/II': 'City of Manila'}, inplace=True)

# Append new row into agshare DF
agshare_by_muni = pd.concat([agshare_by_muni, manila], ignore_index=True)

# Drop all Manila sub-municipalities
drop_indexes = agshare_by_muni.loc[(agshare_by_muni['PROVINCE'] == 39) & (agshare_by_muni['MUN_NAME'] != 'City of Manila')].index
agshare_by_muni.drop(drop_indexes, inplace=True)

agshare_by_muni.loc[agshare_by_muni['PROVINCE'] == 39]

Step 12: Join agshare table with PGSC codes

In [ ]:
# Define filepath to xlsx file with municipality PSGC codes
psgc_fp = r"C:\Users\HOWARD\Desktop\DENR CCIMS\C - VULNERABILITY\Health Vulnerability\ABM3_PercentRural_PSA_2020.xlsx"

# Read excel file as DF
psgc_df = pd.read_excel(psgc_fp, # sheet_name='PSGC',
                        usecols=['ADM2', 'ADM3_EN', 'ADM3_CODE'])

psgc_df.head(10)

Clean up municipality names and codes before joining

In [ ]:
# Clean up data: convert 0 into -
psgc_df['ADM3_EN'] = psgc_df['ADM3_EN'].replace('0', '-', regex=True)

# Remove parentheticals from municipality names
aliases = psgc_df['ADM3_EN'].str.split(' \(', expand=True)
psgc_df['ADM3_EN'] = aliases[0].str.strip()

# Convert muni code to string
psgc_df['ADM3_CODE'] = psgc_df['ADM3_CODE'].astype(str).str.zfill(9)

# Check all rows where code starts with '0215' (Cagayan)
psgc_df.loc[psgc_df['ADM3_CODE'].str.startswith('0215')].head(30)

In [ ]:
# In ADM2, change Cotabato to North Cotabato
psgc_df.loc[psgc_df['ADM2'] == 'Cotabato', 'ADM2'] = 'North Cotabato'

# Change ADM2 of Cotabato City to Cotabato City
psgc_df.loc[psgc_df['ADM3_EN'] == 'City of Cotabato', 'ADM2'] = 'Cotabato City'

# Change ADM2 of City of Manila to City of Manila
psgc_df.loc[psgc_df['ADM3_EN'] == 'City of Manila', 'ADM2'] = 'City of Manila'

# Change ADM3_EN of Sto. Tomas (in Pampanga) to Santo Tomas
psgc_df.loc[psgc_df['ADM3_EN'] == 'Sto. Tomas', 'ADM3_EN'] = 'Santo Tomas'

Perform join

In [ ]:
# Join province PSGC codes to province names
agshare_by_muni_psgc = agshare_by_muni.merge(psgc_df, how='left', 
                                            left_on=[agshare_by_muni['PROV_NAME'].str.lower(), agshare_by_muni['MUN_NAME'].str.lower()],
                                            right_on=[psgc_df['ADM2'].str.lower(), psgc_df['ADM3_EN'].str.lower()])
                                             # left_on='MUN_NAME', 
                                             # right_on='ADM3_EN')

agshare_by_muni_psgc.loc[agshare_by_muni_psgc['PROV_NAME'] == 'CAGAYAN']

In [ ]:
# Check if some rows failed to join
agshare_by_muni_psgc.loc[agshare_by_muni_psgc['ADM3_CODE'].isnull()]#['MUN_NAME'].values

In [ ]:
agshare_by_muni_psgc.columns

In [ ]:
agshare_by_muni_psgc.shape

In [ ]:
agshare_by_muni_psgc['ADM3_CODE_PH'] = 'PH' + agshare_by_muni_psgc['ADM3_CODE']

agshare_by_muni_psgc.head(10)

In [ ]:
# Select only relevant columns for final DF (for export)
final_df = agshare_by_muni_psgc[['REGION', 'PROV_NAME', 'ADM3_EN',
                                 'ADM3_CODE_PH', 'TOTAL_EMPLOYED',
                                 'AGRI_EMPLOYED', 'PERCENT_AGRI']]
# Define dict to rename some columns
col_names = {
    'PROV_NAME': 'ADM2_EN',
    'ADM3_CODE_PH': 'ADM3_CODE',
    'REGION': 'REGION_NO' # 'ADM1_CODE'
}

# Apply renaming dict
final_df = final_df.rename(columns=col_names)

# Convert REGION_NO to int to remove decimal

final_df['REGION_NO'] = final_df['REGION_NO'].astype(int)

# final_df['ADM1_CODE'] = 'PH' + final_df['ADM1_CODE'].astype(int).astype(str).str.zfill(2) + '0000000'

final_df.head(20)

In [ ]:
final_df.to_csv(r'agshare_by_muni.csv', index=False)

Epilogue: calculate mean PERCENT_AGRI per province

In [ ]:
by_prov = final_df.groupby('ADM2_EN', as_index=False).mean(numeric_only=True)

pd.set_option('display.max_rows', None)
by_prov